In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/CS682_Project/

Mounted at /content/drive
/content/drive/MyDrive/CS682_Project


In [ ]:
!pip install googletrans==3.1.0a0

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 1.5 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16352 sha256=a42f370defbea7ddc7c365f37e9f3ef3dc001b511a7366f51ea1535bf04a197c
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Succe

# Imports and Downloads

In [ ]:
import subprocess
import kagglehub
import pandas as pd
from collections import defaultdict, Counter
import os
import subprocess
from tqdm.auto import tqdm

Download Spotify datasets

In [ ]:
file_1 = 'SpotifyDataset/train.csv'
file_2 = 'SpotifyDataset/songs_with_attributes_and_lyrics.csv'

# Download and move the first dataset if it doesn't exist
if not os.path.exists(file_1):
    kagglehub.dataset_download("thedevastator/spotify-tracks-genre-dataset")
    source_path_1 = '/root/.cache/kagglehub/datasets/thedevastator/spotify-tracks-genre-dataset/versions/1/train.csv'
    subprocess.run(['mv', source_path_1, file_1], check=True)
else:
    print(f"{file_1} already exists. Skipping download.")

# Download and move the second dataset if it doesn't exist
if not os.path.exists(file_2):
    kagglehub.dataset_download("bwandowando/spotify-songs-with-attributes-and-lyrics")
    source_path_2 = '/root/.cache/kagglehub/datasets/bwandowando/spotify-songs-with-attributes-and-lyrics/versions/18/songs_with_attributes_and_lyrics.csv/songs_with_attributes_and_lyrics.csv'
    subprocess.run(['mv', source_path_2, file_2], check=True)
else:
    print(f"{file_2} already exists. Skipping download.")

SpotifyDataset/train.csv already exists. Skipping download.
SpotifyDataset/songs_with_attributes_and_lyrics.csv already exists. Skipping download.


In [ ]:
d_genre = pd.read_csv('SpotifyDataset/train.csv').drop_duplicates(subset=['track_name', 'album_name'])
d_lyrics = pd.read_csv('SpotifyDataset/songs_with_attributes_and_lyrics.csv').drop_duplicates(subset=['name', 'album_name'])

song_df_total = pd.merge(d_genre, d_lyrics, left_on=['track_name', 'album_name'], right_on=['name', 'album_name'], how='inner')[['track_name', 'track_genre', 'lyrics']]

In [ ]:
unique_genres = song_df_total['track_genre']
genre_to_idx = {genre: idx for idx, genre in enumerate(unique_genres.unique())}
song_df_total['track_genre_numerical'] = song_df_total['track_genre'].map(genre_to_idx)

# clean lyric data slightly
song_df_total['lyrics'] = song_df_total['lyrics'].apply(lambda x: x.replace('♪', ''))

# song_df_total.to_csv('spotify_data.csv', index= False)

(Roughly) Filter out nonenglish posts

In [ ]:
from googletrans import Translator

translator = Translator()

tqdm.pandas()

def is_english(text):
  try:
    lang = translator.detect(text).lang
    return lang == 'en'
  except Exception as e:
    return False

song_df_total = song_df_total[song_df_total['lyrics'].progress_apply(is_english)].reset_index(drop=True)

  0%|          | 0/30859 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
song_df_total.to_csv('spotify_data_english.csv', index=False)

In [ ]:
banned_genres = ['afrobeat', 'anime', 'brazil', 'british', 'dub', 'cantopop', 'dancehall', 'french', 'german', 'indian', 'iranian', 'j-dance', 'j-idol', 'j-pop', 'j-rock', 'k-pop', 'latino', 'latin', 'malay', 'mandopop', 'mpb', 'opera', 'pagode', 'reggae', 'reggaeton', 'salsa', 'samba', 'ska', 'spanish', 'swedish', 'turkish', 'world-music']
song_df_total_eng = song_df_total[~song_df_total['track_genre'].isin(banned_genres)]

In [ ]:
song_df_total_eng['track_genre'].unique()

array(['acoustic', 'alt-rock', 'alternative', 'ambient', 'black-metal',
       'bluegrass', 'blues', 'breakbeat', 'chicago-house', 'children',
       'chill', 'classical', 'club', 'comedy', 'country', 'dance',
       'dancehall', 'death-metal', 'deep-house', 'detroit-techno',
       'disco', 'disney', 'drum-and-bass', 'dubstep', 'edm', 'electro',
       'electronic', 'emo', 'folk', 'funk', 'garage', 'gospel', 'goth',
       'grindcore', 'groove', 'grunge', 'guitar', 'happy', 'hard-rock',
       'hardcore', 'hardstyle', 'heavy-metal', 'hip-hop', 'honky-tonk',
       'house', 'idm', 'indie-pop', 'indie', 'industrial', 'jazz', 'kids',
       'metal', 'metalcore', 'minimal-techno', 'new-age', 'party',
       'piano', 'pop-film', 'pop', 'power-pop', 'progressive-house',
       'psych-rock', 'punk-rock', 'punk', 'r-n-b', 'rock-n-roll', 'rock',
       'rockabilly', 'sad', 'show-tunes', 'singer-songwriter', 'sleep',
       'soul', 'study', 'synth-pop', 'techno', 'trance', 'trip-hop'],
      dt

In [ ]:
song_df_total_eng['ascii_proportion'] = song_df_total_eng['lyrics'].apply(lambda x: sum([c.isascii() for c in x]) / len(x))

<ipython-input-98-d423ee61c8f0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  song_df_total_eng['ascii_proportion'] = song_df_total_eng['lyrics'].apply(lambda x: sum([c.isascii() for c in x]) / len(x))


In [ ]:
song_df_total_eng_ascii = song_df_total_eng[song_df_total_eng['ascii_proportion'] > 0.99].reset_index(drop=True)

In [ ]:
song_df_total_eng_ascii

,track_name,track_genre,lyrics,track_genre_numerical,ascii_proportion
0,Ghost - Acoustic,acoustic,"Today, I'm kinda feeling like a ghost\n Call m...",0,1.0
1,To Begin Again,acoustic,"When the world was ending, I'd hold you in my ...",0,1.0
2,Can't Help Falling In Love,acoustic,Wise men say\n \n Only fools rush in\n \n But ...,0,1.0
3,Hold On,acoustic,Ripped up sheets of paper\n Laying right next ...,0,1.0
4,Days I Will Remember,acoustic,These are the days I will remember\n These are...,0,1.0
...,...,...,...,...,...
25317,Exordium,trip-hop,There once was a boy who lived in a house with...,110,1.0
25318,Shadow on a Harvest Moon,trip-hop,Let me tell you about this torch I carry\n It'...,110,1.0
25319,"God Is a DJ - Live from Brixton - April 8, 2011",trip-hop,This is my church\n This is where I heal my hu...,110,1.0
25320,My Head Is My Only House Unless It Rains - 201...,trip-hop,I'll let a train be my feet if it's too far to...,110,1.0


In [ ]:
pd.set_option('display.max_rows', 500)
song_df_total_eng_ascii['track_genre'].value_counts()

,count
track_genre,
dance,824
country,762
chill,730
acoustic,662
power-pop,635
deep-house,622
honky-tonk,588
death-metal,583
alt-rock,577


In [ ]:
song_df_total_eng_ascii['track_genre'].unique()

array(['acoustic', 'alt-rock', 'alternative', 'ambient', 'black-metal',
       'bluegrass', 'blues', 'breakbeat', 'chicago-house', 'children',
       'chill', 'classical', 'club', 'comedy', 'country', 'dance',
       'dancehall', 'death-metal', 'deep-house', 'detroit-techno',
       'disco', 'disney', 'drum-and-bass', 'dubstep', 'edm', 'electro',
       'electronic', 'emo', 'folk', 'funk', 'garage', 'gospel', 'goth',
       'grindcore', 'groove', 'grunge', 'guitar', 'happy', 'hard-rock',
       'hardcore', 'hardstyle', 'heavy-metal', 'hip-hop', 'honky-tonk',
       'house', 'idm', 'indie-pop', 'indie', 'industrial', 'jazz', 'kids',
       'metal', 'metalcore', 'minimal-techno', 'new-age', 'party',
       'piano', 'pop-film', 'pop', 'power-pop', 'progressive-house',
       'psych-rock', 'punk-rock', 'punk', 'r-n-b', 'rock-n-roll', 'rock',
       'rockabilly', 'sad', 'show-tunes', 'singer-songwriter', 'sleep',
       'soul', 'study', 'synth-pop', 'techno', 'trance', 'trip-hop'],
      dt

In [ ]:
genre_mapping = {
    'breakbeat': 'ELECTRIC', 'chicago-house': 'ELECTRIC', 'deep-house': 'ELECTRIC', 'detroit-techno': 'ELECTRIC',
    'dubstep': 'ELECTRIC', 'edm': 'ELECTRIC', 'electro': 'ELECTRIC', 'electronic': 'ELECTRIC', 'hardstyle': 'ELECTRIC',
    'house': 'ELECTRIC', 'idm': 'ELECTRIC', 'minimal-techno': 'ELECTRIC', 'progressive-house': 'ELECTRIC',
    'techno': 'ELECTRIC', 'trance': 'ELECTRIC', 'drum-and-bass': 'ELECTRIC',

    'alternative': 'ALT', 'indie': 'ALT',

    'ambient': 'AMBIENT', 'new-age': 'AMBIENT',

    'black-metal': 'HEAVY METAL', 'death-metal': 'HEAVY METAL', 'grindcore': 'HEAVY METAL', 'heavy-metal': 'HEAVY METAL',
    'hardcore': 'HEAVY METAL', 'metal': 'HEAVY METAL', 'metalcore': 'HEAVY METAL', 'industrial': 'HEAVY METAL',

    'r-n-b': 'RNB',

    'alt-rock': 'ROCK', 'rock-n-roll': 'ROCK', 'rock': 'ROCK', 'rockabilly': 'ROCK', 'hard-rock': 'ROCK',
    'psych-rock': 'ROCK', 'punk-rock': 'ROCK', 'grunge': 'ROCK',

    'bluegrass': 'TRADITIONAL', 'blues': 'TRADITIONAL', 'country': 'TRADITIONAL', 'honky-tonk': 'TRADITIONAL',

    'party': 'PARTY', 'dance': 'PARTY', 'club': 'PARTY', 'disco': 'PARTY',

    'indie-pop': 'POP', 'pop': 'POP', 'synth-pop': 'POP', 'power-pop': 'POP', 'pop-film': 'POP',

    'classical': 'CLASSICAL',

    'jazz': 'JAZZ',

    'children': 'KIDS', 'kids': 'KIDS',

    'disney': 'SHOW-TUNES', 'show-tunes': 'SHOW-TUNES',

    'gospel': 'GOSPEL', 'soul': 'GOSPEL',

    'folk': 'FOLK', 'singer-songwriter': 'FOLK',

    'goth': 'REBEL', 'emo': 'REBEL', 'punk': 'REBEL',

    'hip-hop': 'HIP-HOP', 'trip-hop': 'HIP-HOP',

    'funk': 'FUNK',

    'chill': 'CHILL/FOCUS', 'sleep': 'CHILL/FOCUS', 'study': 'CHILL/FOCUS',

    'comedy': 'COMEDY',

    'happy': 'HAPPY',

    'sad': 'SAD',

    'acoustic': 'ACOUSTIC', 'piano': 'ACOUSTIC', 'guitar': 'ACOUSTIC'
}

song_df_total_eng_ascii['track_genre_mapped'] = song_df_total_eng_ascii['track_genre'].map(genre_mapping)

In [ ]:
song_df_total_eng_ascii['track_genre_mapped'].value_counts()

,count
track_genre_mapped,
ELECTRIC,4407
HEAVY METAL,3367
ROCK,2470
TRADITIONAL,2410
POP,1727
PARTY,1679
ACOUSTIC,999
REBEL,956
CHILL/FOCUS,759


In [ ]:
# prompt: from song_df_total_eng_ascii only take electric, heavy metal, rock, traditional, pop, rnb, comedy, funk, ambient, and jazz

selected_genres = ['ELECTRIC', 'HEAVY METAL', 'ROCK', 'TRADITIONAL', 'POP', 'RNB', 'COMEDY', 'FUNK', 'AMBIENT', 'JAZZ']
song_df_selected = song_df_total_eng_ascii[song_df_total_eng_ascii['track_genre_mapped'].isin(selected_genres)]

In [ ]:
unique_genres = selected_genres
genre_to_idx = {genre: idx for idx, genre in enumerate(unique_genres)}
song_df_selected['track_genre_numerical'] = song_df_selected['track_genre_mapped'].map(genre_to_idx)

song_df_selected_out = song_df_selected[['lyrics', 'track_genre_numerical']].rename(columns={'lyrics': 'text', 'track_genre_numerical': 'label'})

<ipython-input-150-4b73041ed6ea>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  song_df_selected['track_genre_numerical'] = song_df_selected['track_genre_mapped'].map(genre_to_idx)


In [ ]:
song_df_selected.to_csv('spotify_umbalanced_10_grouped_genre.csv', index=False)

In [ ]:
train_val = song_df_selected_out.sample(frac=0.9, random_state=682)
# 10% for test
test = song_df_selected_out.drop(train_val.index).reset_index(drop=True)
# 80% for train
train = train_val.sample(frac=8/9, random_state=682)
# 10% for val
val = train_val.drop(train.index).reset_index(drop=True)

In [ ]:
len(train) / len(song_df_selected_out), len(val) / len(song_df_selected_out), len(test) / len(song_df_selected_out)

(0.7999742234824075, 0.10001288825879624, 0.10001288825879624)

In [ ]:
# prompt: output all three with the general title spotify_10_{train/val/test}.csv

train.to_csv('spotify_10/spotify_10_train_unaugmented.csv', index=False)
val.to_csv('spotify_10/spotify_10_val.csv', index=False)
test.to_csv('spotify_10/spotify_10_test.csv', index=False)

# Basic EDA

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english')) | set(stopwords.words('spanish'))

def preprocess_lyrics(lyrics):
    words = lyrics.lower().split()
    filtered_words = [word for word in words if word.isalnum() and word not in stop_words]
    return filtered_words

word_counts_by_genre = defaultdict(Counter)
genre_song_counts = song_df_total['track_genre'].value_counts()


for _, row in song_df_total.iterrows():
    genre = row['track_genre']
    lyrics = row['lyrics']
    words = preprocess_lyrics(lyrics)
    word_counts_by_genre[genre].update(words)


total_word_counts = Counter()
for genre_counts in word_counts_by_genre.values():
    total_word_counts.update(genre_counts)

top_words = total_word_counts.most_common(50)  # Adjust the number of top words as needed


genre_word_proportions = defaultdict(Counter)
for genre, word_counts in word_counts_by_genre.items():
    for word, count in word_counts.items():
        genre_word_proportions[genre][word] = count / genre_song_counts[genre]

for word, _ in top_words:
    print(f"\nWord: {word}")
    for genre in genre_word_proportions:
        if word in genre_word_proportions[genre]:
            print(f"  {genre}: {genre_word_proportions[genre][word]:.4f}")


# Text Augmentation: Backpropogation

In [ ]:
import random
from typing import List, Tuple, Any
from collections import defaultdict
from tqdm.auto import tqdm

class DatasetUpsampler:
    def __init__(self):
        pass

    def _upsample_sentence(self, string: str, num_target_sentences: int, verbose: int = 0) -> List[str]:
        """
        Generate an upsampled list of a given sentence.

        Args:
            string (str): The input sentence to be upsampled.
            num_target_sentences (int): Target number of sentences to generate.

        Returns:
            List[str]: A list containing the upsampled sentences.
        """
        pass  # Define the sentence upsampling logic here

    def _upsample_class(self, string_arr: List[str], num_target_sentences: int, verbose: int = 0) -> List[str]:
        """
        Upsample a list of sentences to reach a target number of sentences for a specific class.

        Args:
            string_arr (List[str]): The list of sentences belonging to a class.
            num_target_sentences (int): Target number of sentences for the class.

        Returns:
            List[str]: A list containing upsampled sentences for the class.
        """
        if num_target_sentences <= len(string_arr):
            return string_arr
        else:
            # Calculate how many sentences are needed per original sentence in string_arr
            sentences_per_string = num_target_sentences // len(string_arr)
            remaining_sentences = num_target_sentences % len(string_arr)

            # Upsample each sentence and aggregate results
            upsampled_sentences = []
            for i, string in (enumerate(string_arr) if verbose == 0 else tqdm(enumerate(string_arr))):
                # Upsample with an extra sentence if needed to reach exact count
                target_count = sentences_per_string + (1 if i < remaining_sentences else 0)
                upsampled_sentences.extend(self._upsample_sentence(string, target_count, verbose))

            # assert len(upsampled_sentences) == num_target_sentences

            return upsampled_sentences

    def upsample_dataset(self, X: List[str], y: List[str], min_sentences_per_class: int, verbose: int = 0) -> Tuple[List[str], List[str]]:
        """
        Upsample a dataset to ensure each class has a minimum number of sentences.

        Args:
            X (List[str]): The list of data items.
            y (List[str]): The list of labels corresponding to each data item in X.
            min_sentences_per_class (int): Minimum number of sentences per class after upsampling.
            verbose (int): Verbosity level (0: none) (1: full)

        Returns:
            Tuple[List[str], List[str]]: Upsampled data (X) and corresponding labels (y).
        """
        if len(X) != len(y):
            raise ValueError("X and y must have the same length.")
        if min_sentences_per_class <= 0:
            raise ValueError("min_sentences_per_class must be a positive integer.")

        # Group X by labels in y
        X_by_y = defaultdict(list)
        for x_item, y_label in zip(X, y):
            X_by_y[y_label].append(x_item)

        # Upsample each class to the minimum required sentences
        for y_label, X_arr in (X_by_y.items() if verbose == 0 else tqdm(X_by_y.items())):
            X_by_y[y_label] = self._upsample_class(X_arr, min_sentences_per_class, verbose)

        # Combine upsampled data and labels
        X_upsampled = []
        y_upsampled = []
        for y_label, X_arr in X_by_y.items():
            X_upsampled.extend(X_arr)
            y_upsampled.extend([y_label] * len(X_arr))

        # Randomize X_upsampled and y_upsampled
        combined = list(zip(X_upsampled, y_upsampled))
        random.shuffle(combined)
        X_upsampled, y_upsampled = zip(*combined)

        return list(X_upsampled), list(y_upsampled)


In [ ]:
# class DuplicateUpsampler(DatasetUpsampler):
#   """
#   Upsample a dataset to ensure each class has a minimum number of sentences via duplication.
#   """
#   def __init__(self):
#     pass

#   def _upsample_sentence(self, string: str, num_target_sentences: int) -> List[str]:
#     """
#     Generate an upsampled list of a given sentence.

#     Args:
#         string (str): The input sentence to be upsampled.
#         num_target_sentences (int): Target number of sentences to generate.
#     """
#     return [string] * num_target_sentences

# d = DuplicateUpsampler()
# lyric_upsampled, genre_upsampled = d.upsample_dataset(song_df_total['lyrics'], song_df_total['track_genre_numerical'], 100)

NameError: name 'song_df_total' is not defined

In [ ]:
import googletrans
class BacktranslateUpsampler(DatasetUpsampler):
  """
  Upsample a dataset to ensure each class has a minimum number of sentences via backtranslation.
  """
  def __init__(self):
    self._translator = googletrans.Translator()

  def _backtranslate_sentence(self, sentence, language):
    """Backtranslates a sentence using Google Translate.

    Args:
      sentence: The sentence to backtranslate.
      language: The language to backtranslate to.

    Returns:
      The backtranslated sentence.
    """
    translated_sentence = self._translator.translate(sentence, dest=language).text
    backtranslated_sentence = self._translator.translate(translated_sentence, dest='en').text
    backtranslated_sentence = backtranslated_sentence.replace('♪', '')

    original_ascii_ratio = sum([c.isascii() for c in sentence]) / len(sentence)
    back_ascii_ratio = sum([c.isascii() for c in backtranslated_sentence]) / len(backtranslated_sentence)

    eps = 0.01

    # heuristic: is the newly translated sentence less ascii than before?
    if back_ascii_ratio > original_ascii_ratio - eps:
      return backtranslated_sentence
    else:
      raise Exception(f'Error backtranslating to {language}')

  def _upsample_sentence(self, sentence: str, num_target_sentences: int, verbose: int = 0) -> List[str]:
    new_sentences = set()
    new_sentences.add(sentence)
    languages = list(googletrans.LANGUAGES.keys())
    random.shuffle(languages)

    # upsample
    for lang in languages:
      try:
        b_sent = self._backtranslate_sentence(sentence, lang)
        new_sentences.add(b_sent)
      except:
        if verbose == 2:
          print(f'Error backtranslating to {lang}')
      if len(new_sentences) >= num_target_sentences:
        break

    # Translate back to English
    return list(new_sentences)

  # def _upsample_sentence(self, string: str, num_target_sentences: int) -> List[str]:
  #   """
  #   Generate an upsampled list of a given sentence.

  #   Args:
  #       string (str): The input sentence to be upsampled.
  #       num_target_sentences (int): Target number of sentences to generate.
  #   """
  #   return list(backtranslate(string))

Upsample Spotify Dataset

In [ ]:
train_df = pd.read_csv('spotify_10/spotify_10_train_unaugmented.csv')
X_train = train_df['text']
y_train = train_df['label']

d = BacktranslateUpsampler()
X_train_upsampled, y_train_upsampled = d.upsample_dataset(X_train, y_train, 1000, verbose=1)

  0%|          | 0/10 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
train_augmented_df = pd.DataFrame({'text': X_train_upsampled, 'label': y_train_upsampled})
train_augmented_df.to_csv('spotify_10/spotify_10_train_augmented.csv', index=False)

NameError: name 'X_train_upsampled' is not defined

In [ ]:
train_augmented_df['label'].value_counts()

,count
label,
0,3529
1,2683
2,1997
3,1935
4,1365
8,1000
6,1000
5,1000
9,1000


Upsample Linkedin Dataset

In [ ]:
train_df = pd.read_csv('LinkedInDataset/linkedin_train.csv')
X_train = train_df['description']
y_train = train_df['title']

d = BacktranslateUpsampler()
X_train_upsampled, y_train_upsampled = d.upsample_dataset(X_train, y_train, 1000, verbose=1)

  0%|          | 0/10 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
train_augmented_df = pd.DataFrame({'description': X_train_upsampled, 'title': y_train_upsampled})
train_augmented_df.to_csv('LinkedInDataset/linkedin_train_backtranslated_augmented.csv', index=False)

In [ ]:
train_augmented_df['title'].value_counts()

,count
title,
developer,2432
nurse,2389
sales,2371
customer service,1494
accountant,1421
project manager,1372
therapist,1223
human resources,1152
teacher,1109
